# Fundamental tensor decompositions.
### Last modification (13.05.2018)


In this tutorial we provide a theoretical backgound on the fundamental tensor
decompositions of multidimensional arrays and show how these data algorithms
can be used with [hottbox](https://github.com/hottbox/hottbox) through **CPD**, **HOSVD**, **HOOI** and **TTSVD** classes.

More details on **CPD**, **HOSVD**, **HOOI** and **TTSVD** classes can be found on the [documentation page](https://hottbox.github.io/stable/api/hottbox.algorithms.decomposition).

> **Note:** this tutorial assumes that you are familiar with the basics of 
tensor algebra, tensor representaitons in different forms and the corresponding conventional 
notation. If you are new to these topics, check out our previous tutorials:
[tutorial_1](https://github.com/hottbox/hottbox-tutorials/blob/master/1_N-dimensional_arrays_and_Tensor_class.ipynb) and 
[tutorial_2](https://github.com/hottbox/hottbox-tutorials/blob/master/2_Efficient_representations_of_tensors.ipynb).

**Requirements:** ``hottbox>=0.1.2``

**Author:** Ilya Kisil - ilyakisil@gmail.com

In [1]:
import numpy as np
from hottbox.core import Tensor, residual_tensor
from hottbox.algorithms.decomposition import TTSVD, HOSVD, HOOI, CPD
from hottbox.metrics import residual_rel_error

# Fundamental tensor decompositions and their implementation

The following algorithms have been implemented in **``hottbox>=0.1.2``**:

- CPD: produces instance of **TensorCPD** class
- HOSVD: produces instance of **TensorTKD** class
- HOOI: produces instance of **TensorTKD** class
- TTSVD: produces instance of **TensorTT** class

> **Note:** more background is coming soon

In this tutorial we use the following randomly generated $3$-rd order tensor for our examples.

In [2]:
np.random.seed(0)
I, J, K = 5, 6, 7

# array_3d = np.arange(I * J * K).reshape((I, J, K)).astype(np.float)
array_3d = np.random.rand(I * J * K).reshape((I, J, K)).astype(np.float)

tensor = Tensor(array_3d)

# Canonical Polyadic Decomposition (CPD)
![tensorcpd](./images/TensorCPD.png)

The CPD decomposition (also referred to as PARAFAC or CANDECOMP) factorizes an $N$-th order tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I_1 \times I_2 \times \cdots \times I_N}$ into a linear combination of terms $\mathbf{b}_r^{(1)} \circ \mathbf{b}_r^{(2)} \circ \cdots \circ \mathbf{b}_r^{(N)}$, which are rank-$1$ tensors. In other words the tensor $\mathbf{\underline{X}}$ is decomposed as

\begin{equation}
\begin{aligned}
\mathbf{\underline{X}} & \simeq \sum_{r=1}^{R} \lambda_r \mathbf{b}_r^{(1)} \circ \mathbf{b}_r^{(2)} \circ \cdots \circ \mathbf{b}_r^{(N)}\\
& = \mathbf{\underline{\Lambda}} \times_1 \mathbf{\underline{B}}^{(1)} \times_2 \mathbf{\underline{B}}^{(2)} \cdots \times_N \mathbf{\underline{B}}^{(N)} \\
& = \Big[    \mathbf{\underline{\Lambda}} ;  \mathbf{\underline{B}}^{(1)} ,  \mathbf{\underline{B}}^{(2)}, \dots, \mathbf{\underline{B}}^{(N)}         \Big]
\end{aligned}
\end{equation}
where $\mathbf{\underline{B}^{(N)}}  = \Big[    \mathbf{b}^{(n)}_1 \hspace{2mm} \mathbf{b}^{(n)}_2  \cdots \mathbf{b}^{(n)}_R   \Big] $ i.e. the concatenation of the corresponding vectors. In case of $3$-rd order tensors, for convention, we let $ \mathbf{A} \colon = \mathbf{B}^{(1)}$,  $\mathbf{B} \colon = \mathbf{B}^{(2)}$, $\mathbf{C} \colon = \mathbf{B}^{(3)}$. $\mathbf{\underline{\Lambda}}$ is an $N$-th order tensor having $\lambda_r$ as entries in positions $\mathbf{\underline{\Lambda}}(i_1, i_2, \dots, i_N)$, where $i_1 = i_2 = \cdots = i_N$, and zeroes elsewhere. 


In **``hottbox``** this form is available through the **``TensorCPD``** class.
In order to create such object, you have 2 options:


1) (See $\textit{Efficient representation of multidimensional arrays}$
) Pass a list of factor matrices (2d ``numpy`` arrays) and a vector of values (as 1d ``numpy`` array) for the main diagonal:

```python
tensor_cpd = TensorCPD(fmat=[A, B, C], core_values=values)
```

2) Decompose an original tensor using class CPD

In this tutorial, we focus on point (2).


To decompose a tensor using the CPD decomposition we create an instance of the CPD class and set a Kruskal rank $R$. The Kruskal rank is passed as a tuple so to keep the same format with other tensor decompositions.

In [3]:
cpd = CPD()
R = (5,)

tensor_cpd = cpd.decompose(tensor, R)
type(tensor_cpd)

hottbox.core.structures.TensorCPD

A **``TensorCPD``** object contains the $\mathbf{\underline{\Lambda}}$ values stored in property **``core``**, while the factor matrices $\mathbf{B}^{(n)}$ are stored in property **``fmat``**.

In [13]:
print('Factor matrices')
for mode, fmat in enumerate(tensor_cpd.fmat):
    print('Mode-{} factor matrix is of shape {}'.format(mode, fmat.shape))
    
print('\nCore tensor')
print(type(tensor_cpd.core))
tensor_cpd.core.describe()
tensor_cpd.core.data

Factor matrices
Mode-0 factor matrix is of shape (5, 5)
Mode-1 factor matrix is of shape (6, 5)
Mode-2 factor matrix is of shape (7, 5)

Core tensor
<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 125 elements and its Frobenious norm = 2.24.
Sizes and names of its modes are (5, 5, 5) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[1., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 1., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 1., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 1., 0.],
        [0., 0., 0., 0., 0.]],

       [[0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 0.],
        [0., 0., 0., 0., 1.]]])

In order to convert **``TensorCPD``** into the full representation, simply call: 

```python
tensor_cpd.reconstruct
```

This return an object of the **``Tensor``** class with N-dimensional array calculated as 
described above and being assinged to the **``_data``** attibute.

In [15]:
tensor_full = tensor_cpd.reconstruct

print(type(tensor_full))
tensor_full.describe()
tensor_full.data

<class 'hottbox.core.structures.Tensor'>
This tensor is of order 3, consists of 210 elements and its Frobenious norm = 7.89.
Sizes and names of its modes are (5, 6, 7) and {0: 'mode-0', 1: 'mode-1', 2: 'mode-2'} respectively.


array([[[ 0.4601365 ,  0.38423614,  0.48859033,  0.62131001,
          0.54691852,  0.58361213,  0.49159456],
        [ 0.67292687,  0.59403672,  0.72678594,  0.81013743,
          0.55605843,  0.32617485,  0.70367859],
        [ 0.18032852,  0.2861196 ,  0.27910407,  0.76039979,
          0.48426828,  0.95862907,  0.76088009],
        [ 0.94275828,  0.47486318,  0.84605741,  0.22145674,
          0.74691123,  0.33027761,  0.85913182],
        [ 0.45441219,  0.43541838,  0.44517452,  0.96472375,
          0.58478457,  0.73463438,  0.17714534],
        [ 0.45427474,  0.46247196,  0.48368908,  0.86823801,
          0.42910584,  0.48006745,  0.61487742]],

       [[ 0.47081601,  0.39236953,  0.48652466,  0.50159374,
          0.34103943,  0.12672599,  0.42371199],
        [ 0.29345678,  0.40894483,  0.37908318,  0.78503303,
          0.27212222,  0.27169589,  0.40563341],
        [ 0.4420933 ,  0.29975187,  0.45217461,  0.22632031,
          0.27859889,  0.13801009,  0.94171161],
        

The **``TensorCPD``** object also contains general information about the underlying tensor, such as its **``rank``** and **``order``**.

> **Note:** The **``rank``** is returned as a tuple. Select its first element to have it as an integer.


In [23]:
R = tensor_cpd.rank
N = tensor_cpd.order

print('The rank of the underlying tensor is {}, and the order is {}'.format(R[0],N))

The rank of the underlying tensor is 5, and the order is 3


# Tucker form and **TensorTKD** class

![tensortkd](./images/TensorTKD.png)

For a tensor $\mathbf{\underline{X}} \in \mathbb{R}^{I \times J \times K}$ illustrated above, the tucker form
represents it as a dense core tensor $\mathbf{\underline{G}}$ with multi-linear rank ($Q, R, P$) and a set of
factor matrices $\mathbf{A} \in \mathbb{R}^{I \times Q}, \mathbf{B} \in \mathbb{R}^{J \times R}$ and $\mathbf{C} \in
\mathbb{R}^{K \times P}$.

The tucker form of a tensor is closely related to the CP form and can be expressed through a 
sequence of mode-$n$ products in a similar way.

$$
\mathbf{\underline{X}} = \mathbf{\underline{G}} \times_1 \mathbf{A} \times_2 \mathbf{B} \times_3 \mathbf{C} = \Big[\mathbf{\underline{G}}; \mathbf{A}, \mathbf{B}, \mathbf{C} \Big]
$$

In **``hottbox``** this form is available through the **``TensorTKD``** class.
In order to create such object from its factor matrices and core tenosr, you need to pass a list of factor matrices (2d ``numpy`` arrays) and values for the core tensor (as n-dimensional ``numpy`` array)

```python
tensor_tkd = TensorTKD(fmat=[A, B, C], core_values=values)
```

See **Efficient representation of multidimensional arrays** for further details on this approach.


## Tucker representation through Higher Order Sindular Value Decomposition (HOSVD)

In [4]:
hosvd = HOSVD()
ml_rank = (4,5,6)

tensor_tkd_hosvd = hosvd.decompose(tensor, ml_rank)
type(tensor_tkd_hosvd)

hottbox.core.structures.TensorTKD

## Tucker representation through Higher Order Orthogonal Iteration (HOOI)

In [5]:
hooi = HOOI()
ml_rank = (4,5,6)

tensor_tkd_hooi = hosvd.decompose(tensor, ml_rank)
type(tensor_tkd_hooi)

hottbox.core.structures.TensorTKD

## Tensor Train Decomposition via SVD

In [6]:
tt = TTSVD()
tt_rank = (2,3)

tensor_tt = tt.decompose(tensor, tt_rank)
type(tensor_tt)

hottbox.core.structures.TensorTT

## Evaluating results of tensor decompositions

For each result of the tensor decomposition we can compute a residual tensor and calculate relative error of approximation:
```python
    tensor_res = residual_tensor(tensor, tensor_cpd)
    rel_error = tensor_res.frob_norm / tensor.frob_norm        
```
Or can do it in one line:
```python
    rel_error = residual_rel_error(tensor, tensor_cpd)
```



In [7]:
tensor_cpd_res = residual_tensor(tensor, tensor_cpd)
print('Residual tensor is instance of {}'.format(type(tensor_cpd_res)))

Residual tensor is instance of <class 'hottbox.core.structures.Tensor'>


In [8]:
rel_error = tensor_cpd_res.frob_norm / tensor.frob_norm 
print('Relative error of CPD approximation = {:.2f}'.format(rel_error))

rel_error = residual_rel_error(tensor, tensor_cpd)
print('Relative error of CPD approximation = {:.2f}'.format(rel_error))

Relative error of CPD approximation = 0.31
Relative error of CPD approximation = 0.31


In [9]:
rel_error = residual_rel_error(tensor, tensor_tkd_hosvd)
print('Relative error of HOSVD approximation = {:.2f}'.format(rel_error))

Relative error of HOSVD approximation = 0.21


In [10]:
rel_error = residual_rel_error(tensor, tensor_tkd_hooi)
print('Relative error of HOOI approximation = {:.2f}'.format(rel_error))

Relative error of HOOI approximation = 0.21


In [11]:
rel_error = residual_rel_error(tensor, tensor_tt)
print('Relative error of TT approximation = {:.2f}'.format(rel_error))

Relative error of TT approximation = 0.39
